In [1]:
import jax
import jax.numpy as jnp  # JAX NumPy
import optax  # Optimizers
import equinox as eqx  # Equinox
from diffrax import Dopri5, diffeqsolve, ODETerm, SaveAt

from flowMC.resource.model.common import MLP
from flowMC.resource.model.flowmatching.base import Solver


In [2]:
model = MLP([3,16,16,2], jax.random.PRNGKey(0), activation=jax.nn.swish)
# solver = Dopri5()
# def wrapper(t, y, args):
#     t = jnp.expand_dims(t, axis=-1)  # Ensure t is a column vector
#     input = jnp.concatenate([y, t], axis=-1)
#     return model(input)
# term = ODETerm(wrapper)
solver = Solver(model)

In [5]:
y = solver.sample(
    jax.random.PRNGKey(0),
    10)

In [6]:
y

Array([[ 1.6052856 ,  1.795202  ],
       [-0.45093694, -0.30864018],
       [ 0.1587591 , -1.2021123 ],
       [-0.5126471 ,  0.26435208],
       [ 0.64701885, -1.1801922 ],
       [ 2.1622145 , -2.1851885 ],
       [ 0.34122846, -0.07223853],
       [ 1.2597322 ,  1.28041   ],
       [ 0.9533114 ,  0.36956397],
       [ 0.0073776 , -2.1464915 ]], dtype=float32)